## PART 2: Remove trend and seasonality by least-squares modeling

This script aims to:
- Least squares model to remove the tidal and seasonal signals as well as co-seismic jumps
- Earthquakes selected are obtained from wilber 3 web interface
- Co-seismic jumps from selected earthquakes (>5.1 Mw) 

### Required:
- python
- pandas
- jupyter
- notebook
- matplotlib
- seaborn

this should be easy to set up in a conda env: conda create -c conda-forge -n dtwclustering python=3.7 pandas numpy jupyter notebook matplotlib seaborn

__Author: Utpal Kumar @Institute of Earth Sciences, Academia Sinica__

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import least_squares as ls
from numpy import linalg as LA
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dtwclustering.analysis_support import toYearFraction as tyf
from dtwclustering.leastSquareModeling import lsqmodeling

import matplotlib
from matplotlib import style
import scipy.io as sio
import os, glob
from IPython.display import Image, display
from random import randrange 

In [2]:
## default matplotlib parameters
import matplotlib
font = {'family' : 'Times',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)
plt.rcParams["figure.figsize"] = (12, 6)
plt.style.use('ggplot')

### Load data

In [3]:
#load pickle data
dataloc = "pickleFiles"
dUU=pd.read_pickle(os.path.join(dataloc,"dU_data.pickle"))
dNN=pd.read_pickle(os.path.join(dataloc,"dN_data.pickle"))
dEE=pd.read_pickle(os.path.join(dataloc,"dE_data.pickle"))
dUU.head()

,GUKN_U,SANL_U,GS26_U,GS32_U,GS33_U,GS27_U,NAAO_U,PEPU_U,SJPU_U,GS31_U,...,GS15_U,GS28_U,TTUN_U,DNAN_U,MESN_U,W029_U,GS16_U,WULU_U,NSHE_U,DOSH_U
Time,,,,,,,,,,,,,,,,,,,,,
2007-01-01,-3.6104,-21.535,-11.6553,-4.5357,2.7128,6.5048,15.7129,10.9418,-30.4328,8.5945,...,-10.2284,11.2566,30.0541,75.6983,-4.4475,-4.054,-6.0754,-60.0987,4.3838,-22.1776
2007-01-02,4.3376,-19.425,-5.1123,-3.3467,6.4358,11.1978,35.5059,6.8598,-27.1238,7.3335,...,-5.4244,19.1496,29.7561,79.6213,-15.8095,-6.475,-12.6834,-49.5617,10.9828,-11.7836
2007-01-03,13.0606,-15.892,-2.9963,-2.4947,7.1608,7.1998,23.5059,8.8988,-14.8608,7.1765,...,-3.2764,25.5186,34.5391,81.3833,-13.0965,9.169,-4.4514,-53.3637,5.0418,-25.0546
2007-01-04,0.3386,-23.530,-6.6133,-9.6667,2.1708,10.9468,27.1809,17.0198,-36.1068,4.6815,...,-10.8944,12.7456,23.0081,75.2663,-21.1925,-12.239,-4.4514,-61.2687,1.3368,-21.9806
2007-01-05,-3.1744,-21.213,-13.0963,-10.5357,-0.1872,6.2038,21.3349,13.4428,-29.7658,4.0915,...,-5.9304,9.0336,31.5471,76.9883,-40.2215,-9.293,0.2166,-85.4437,0.2348,-20.8926


##  Least square modeling for all the stations for three-components

### Remove seasonlity only (keep the trend and co-seismic jumps)

In [4]:
if not os.path.exists(os.path.join(dataloc,"dU_wo_seasn.pickle")) or not os.path.exists(os.path.join(dataloc,"dN_wo_seasn.pickle")) or not os.path.exists(os.path.join(dataloc,"dE_wo_seasn.pickle")):
    final_dU, final_dN, final_dE = lsqmodeling(dUU, dNN, dEE,stnlocfile="helper_files/stn_loc.txt",  plot_results=False, remove_trend=False, remove_seasonality=True, remove_jumps=False)
else:
    final_dU=pd.read_pickle(os.path.join(dataloc,"dU_wo_seasn.pickle"))
    final_dN=pd.read_pickle(os.path.join(dataloc,"dN_wo_seasn.pickle"))
    final_dE=pd.read_pickle(os.path.join(dataloc,"dE_wo_seasn.pickle"))
    final_dU.head()

Finished LSQ (U): WULU: : 115it [01:37, 10.58it/s]
Finished LSQ (N): SHJU: : 115it [01:54,  1.00it/s]
Finished LSQ (E): GS16: : 115it [02:08,  8.70it/s]


## Check the least square modeling results

In [5]:
all_U_png = glob.glob(os.path.join("LSQOut","*_U.png"))
all_N_png = glob.glob(os.path.join("LSQOut","*_N.png"))
all_E_png = glob.glob(os.path.join("LSQOut","*_E.png"))


if all_U_png:
    display(Image(filename=all_U_png[randrange(0, len(all_U_png))]))     

In [6]:
if all_N_png:
    display(Image(filename=all_N_png[randrange(0, len(all_N_png))])) 

In [7]:
if all_E_png:
    display(Image(filename=all_E_png[randrange(0, len(all_E_png))])) 

## Save lsq modeled result in pickle and mat format

In [8]:
if not os.path.exists(os.path.join(dataloc,"dU_wo_seasn.pickle")) or not os.path.exists(os.path.join(dataloc,"dN_wo_seasn.pickle")) or not os.path.exists(os.path.join(dataloc,"dE_wo_seasn.pickle")):

    final_dU.to_pickle(os.path.join(dataloc,"dU_wo_seasn.pickle"))
    final_dN.to_pickle(os.path.join(dataloc,"dN_wo_seasn.pickle"))
    final_dE.to_pickle(os.path.join(dataloc,"dE_wo_seasn.pickle"))

# read station information
station_loc_file="helper_files/stn_loc.txt"
stnloc = pd.read_csv(station_loc_file, header=None,
                          sep='\s+', names=['stn', 'lon', 'lat'])
stnloc.set_index('stn', inplace=True)
new_stn_info_list = []
for stn in final_dU.columns.values:
    stn = stn.split("_")[0]
    _dict={}
    _dict['stn'] = stn
    _dict['lon'] = stnloc.loc[stn,'lon']
    _dict['lat'] = stnloc.loc[stn,'lat']
    new_stn_info_list.append(_dict)
    
stn_info_df = pd.DataFrame(new_stn_info_list)
stn_info_df.head()
if not os.path.exists(os.path.join(dataloc,'all_data_wo_seasn.mat')):
    sio.savemat(os.path.join(dataloc,'all_data_wo_seasn.mat'), {'slat': stn_info_df['lat'].values, 'slon': stn_info_df['lon'].values, 'tdata': final_dU.index.values,
                                        'stns': stn_info_df['stn'].values, 'dU': np.array(final_dU), 'dN': np.array(final_dN), 'dE': np.array(final_dE)})

# # Saving the slope info in the mat file
# stn_slope = pd.read_csv('stn_slope_res_U.txt', sep='\s+',
#                         header=None, names=['stn', 'lon', 'lat', 'slope'])
# sio.savemat('slope_info.mat', {'slat': stn_slope['lat'].values, 'slon': stn_slope['lon'].values,
#                                'stns': stn_slope['stn'].values, 'slope': stn_slope['slope'].values})


### Compute dN, dE and dU without seasonality, trend and jumps
- to compute the CME (common mode error)

In [9]:
if not os.path.exists(os.path.join(dataloc,"dU_wo_seasn_trend_jump.pickle")) or not os.path.exists(os.path.join(dataloc,"dN_wo_seasn_trend_jump.pickle")) or not os.path.exists(os.path.join(dataloc,"dE_wo_seasn_trend_jump.pickle")):
    final_dU_2, final_dN_2, final_dE_2 = lsqmodeling(dUU, dNN, dEE,stnlocfile="helper_files/stn_loc.txt",  plot_results=False, remove_trend=True, remove_seasonality=True, remove_jumps=True)
else:
    final_dU_2=pd.read_pickle(os.path.join(dataloc,"dU_wo_seasn_trend_jump.pickle"))
    final_dN_2=pd.read_pickle(os.path.join(dataloc,"dN_wo_seasn_trend_jump.pickle"))
    final_dE_2=pd.read_pickle(os.path.join(dataloc,"dE_wo_seasn_trend_jump.pickle"))
    final_dU_2.head()

Finished LSQ (U): HUWE: : 115it [01:43,  1.11it/s]
Finished LSQ (N): SHJU: : 115it [02:00, 12.34it/s]
Finished LSQ (E): GS16: : 115it [02:12, 11.34it/s]


In [10]:
if not os.path.exists(os.path.join(dataloc,"dU_wo_seasn_trend_jump.pickle")) or not os.path.exists(os.path.join(dataloc,"dN_wo_seasn_trend_jump.pickle")) or not os.path.exists(os.path.join(dataloc,"dE_wo_seasn_trend_jump.pickle")):
    final_dU_2.to_pickle(os.path.join(dataloc,"dU_wo_seasn_trend_jump.pickle"))
    final_dN_2.to_pickle(os.path.join(dataloc,"dN_wo_seasn_trend_jump.pickle"))
    final_dE_2.to_pickle(os.path.join(dataloc,"dE_wo_seasn_trend_jump.pickle"))

# read station information
station_loc_file="helper_files/stn_loc.txt"
stnloc = pd.read_csv(station_loc_file, header=None,
                          sep='\s+', names=['stn', 'lon', 'lat'])
stnloc.set_index('stn', inplace=True)
new_stn_info_list = []

# east_coast_stations = ['PEPU', 'DAJN', 'NDHU', 'CHUN', 'SHUL', 'TUNH', 'DAWU', 'CHGO', 'YENL', 'SHAN', 'SOFN', 'TAPE', 'ERPN', 'CHEN', 'TAPO', 'SINL', 'LONT', 'JULI', 'JSUI', 'TTUN', 'NAAO', 'SPAO', 'MOTN', 'SLNP', 'WARO', 'SLIN', 'WULU']
for stn in final_dU_2.columns.values:
    stn = stn.split("_")[0]
    
    _dict={}
    _dict['stn'] = stn
    _dict['lon'] = stnloc.loc[stn,'lon']
    _dict['lat'] = stnloc.loc[stn,'lat']
    new_stn_info_list.append(_dict)
    
stn_info_df = pd.DataFrame(new_stn_info_list)
stn_info_df.head()
stn_info_df.to_csv('helper_files/selected_stations_info.txt', index=False)
if not os.path.exists(os.path.join(dataloc,'all_data_wo_seasn_trend_jump.mat')):
    sio.savemat(os.path.join(dataloc,'all_data_wo_seasn_trend_jump.mat'), {'slat': stn_info_df['lat'].values, 'slon': stn_info_df['lon'].values, 'tdata': final_dU_2.index.values,
                                        'stns': stn_info_df['stn'].values, 'dU': np.array(final_dU_2), 'dN': np.array(final_dN_2), 'dE': np.array(final_dE_2)})
